<a href="https://colab.research.google.com/github/promalpakohz/Kikoyi-Sales-Predictions/blob/main/Kikoyi_00779.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# %%
import pandas as pd
import matplotlib.pyplot as plt
!pip install numpy as np
!pip install pystan
!pip install pandas
!pip install cmdstanpy
# Install a compatible version of numpy before installing prophet
!pip install prophet
from prophet.diagnostics import performance_metrics, cross_validation
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [3]:
df=pd.read_csv('Kikoyis 00779.csv')
df.head(10)

,Item,Year,Month,Bales Sold
0,TPG00779,2021,Jan,2
1,TPG00779,2021,Feb,8
2,TPG00779,2021,Mar,15
3,TPG00779,2021,Apr,10
4,TPG00779,2021,May,6
5,TPG00779,2021,Jun,12
6,TPG00779,2021,Jul,0
7,TPG00779,2021,Aug,1
8,TPG00779,2021,Sept,29
9,TPG00779,2021,Oct,5


In [4]:
#convert month nd year into a string
df['Date']=df['Year'].astype(str) + '-' + df['Month'].astype(str)

In [5]:
#convert the Date into a
df['Date']=pd.to_datetime(df['Date'], format ='mixed')

In [6]:
df=df.rename(columns={'Date':'ds', 'Bales Sold':'y'})
df['ds']=pd.to_datetime(df['ds'])
print(df.head(10))

       Item  Year Month   y         ds
0  TPG00779  2021   Jan   2 2021-01-01
1  TPG00779  2021   Feb   8 2021-02-01
2  TPG00779  2021   Mar  15 2021-03-01
3  TPG00779  2021   Apr  10 2021-04-01
4  TPG00779  2021   May   6 2021-05-01
5  TPG00779  2021   Jun  12 2021-06-01
6  TPG00779  2021   Jul   0 2021-07-01
7  TPG00779  2021   Aug   1 2021-08-01
8  TPG00779  2021  Sept  29 2021-09-01
9  TPG00779  2021   Oct   5 2021-10-01


In [22]:
model= Prophet(
    growth='linear',
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    changepoint_prior_scale=0.15,
    seasonality_mode='additive',
    uncertainty_samples=1000
)

In [23]:
#backtesting configuration
!pip install prophet --upgrade
from prophet.diagnostics import performance_metrics, cross_validation
# Replace cross_validate with manual implementation:
def custom_cross_validate(model, horizon='365 days', period='180 days'):
    df_cv = cross_validation(model, horizon=horizon, period=period)
    df_p = performance_metrics(df_cv)
    return df_p

In [24]:
#custom quarterly pattern(3-month cycles)
model.add_seasonality(
    name='quarterly',
    period=91.25,
    fourier_order=3,
    mode='additive'
)
#conditional monthly effects
model.add_regressor('month_num')
#  Create month numbers in BOTH dataframes
df['month_num'] = df['ds'].dt.month

In [25]:
#fitting the model
model.fit(df)
#creating 2025 prediction framework
future_2025= model.make_future_dataframe(
    periods=12,
    freq='MS',
    include_history=False
)
# Add month numbers
future_2025['month_num']= future_2025['ds'].dt.month
#Generate predictions
forecast=model.predict(future_2025)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpuv7a_n0e/u7xy587y.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpuv7a_n0e/yzqzx9v7.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=65495', 'data', 'file=/tmp/tmpuv7a_n0e/u7xy587y.json', 'init=/tmp/tmpuv7a_n0e/yzqzx9v7.json', 'output', 'file=/tmp/tmpuv7a_n0e/prophet_modelbvs85t2w/prophet_model-20250516091103.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
09:11:03 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:11:03 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [26]:
print(forecast)

           ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0  2025-01-01 -136.938410   -3.634879    9.170157  -136.938411  -136.938410   
1  2025-02-01 -136.441045    0.153783   13.330982  -136.441045  -136.441044   
2  2025-03-01 -135.991811   15.557832   28.763472  -135.991812  -135.991810   
3  2025-04-01 -135.494445   15.636476   28.658861  -135.494446  -135.494443   
4  2025-05-01 -135.013123   24.924129   37.743352  -135.013125  -135.013121   
5  2025-06-01 -134.515757   13.787250   26.671327  -134.515760  -134.515754   
6  2025-07-01 -134.034435    8.743790   22.197113  -134.034439  -134.034431   
7  2025-08-01 -133.537069   11.924590   24.437156  -133.537073  -133.537064   
8  2025-09-01 -133.039703   14.322138   27.064247  -133.039708  -133.039697   
9  2025-10-01 -132.558381    8.842300   22.626965  -132.558387  -132.558374   
10 2025-11-01 -132.061015    3.304095   15.870225  -132.061022  -132.061007   
11 2025-12-01 -131.579693   -2.444223   10.204916  -

In [33]:

final_output = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
final_output['Year'] = final_output.ds.dt.year
final_output['Month'] = final_output.ds.dt.strftime('%b')
final_output['Confidence Interval'] = final_output.apply(
    lambda x: f"±{(x.yhat_upper - x.yhat_lower)/2:.1f}",
    axis=1
)



In [34]:
import plotly.express as px

fig = px.line(forecast, x='ds', y='yhat',
              title='2025 Sales Forecast ')
fig.add_scatter(x=df.ds, y=df.y, mode='lines', name='Historical')
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Bales Sold',
    hovermode='x unified'
)
fig.show()
